In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge


In [2]:
# Load a dataset
df = pd.read_csv('gorilla_tug_of_war.csv')

# Display first few rows
df.head()

,WHT,FRC,AGE,DSI,SUS,GND,HMNS
0,199.4,145,40,603,Western Lowland,Male,224
1,197.7,146,22,268,Western Lowland,Male,220
2,136.0,117,30,47,Western Lowland,Female,144
3,138.0,100,36,52,Western Lowland,Female,174
4,196.2,102,35,488,Mountain,Male,216


In [3]:
# Separate features and target variable:
X = df.drop(columns=['HMNS'])
y = df['HMNS']

In [4]:
# Separate features into numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [5]:
# Split into train/test sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Copies of original train and test sets
X_train_orig = X_train.copy()
X_test_orig = X_test.copy()

In [7]:
# Remove outliers
def remove_outliers_iqr(df, numerical_cols):
    df_filtered = df.copy()
    for col in numerical_cols:
        df_filtered[col] = df_filtered[col].astype(float)
        Q1 = df_filtered[col].quantile(0.25)
        Q3 = df_filtered[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_filtered = df_filtered[(df_filtered[col] >= lower_bound) & (df_filtered[col] <= upper_bound)]
    return df_filtered

X_train_filtered = remove_outliers_iqr(X_train, numerical_cols)
y_train_filtered = y_train.loc[X_train_filtered.index]

X_test_filtered = remove_outliers_iqr(X_test, numerical_cols)
y_test_filtered = y_test.loc[X_test_filtered.index]


In [8]:
# One-hot encoding
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_cat_filtered = X_train_filtered[categorical_cols]
X_train_encoded = pd.DataFrame(
    encoder.fit_transform(X_train_cat_filtered),
    columns=encoder.get_feature_names_out(categorical_cols),
    index=X_train_cat_filtered.index
)

X_test_cat_filtered = X_test_filtered[categorical_cols]
X_test_encoded = pd.DataFrame(
    encoder.transform(X_test_cat_filtered),
    columns=encoder.get_feature_names_out(categorical_cols),
    index=X_test_cat_filtered.index
)

X_train_final = pd.concat([X_train_filtered[numerical_cols], X_train_encoded], axis=1)
X_test_final = pd.concat([X_test_filtered[numerical_cols], X_test_encoded], axis=1)

X_train_final.head()

,WHT,FRC,AGE,DSI,SUS_Cross River,SUS_Grauer's,SUS_Mountain,SUS_Western Lowland,GND_Female,GND_Male
249,177.0,118.0,28.0,184.0,0.0,0.0,0.0,1.0,0.0,1.0
433,197.9,118.0,25.0,157.0,0.0,0.0,1.0,0.0,0.0,1.0
19,152.4,102.0,38.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
301,155.4,70.0,35.0,455.0,1.0,0.0,0.0,0.0,1.0,0.0
229,124.8,81.0,24.0,19.0,0.0,0.0,0.0,1.0,1.0,0.0


In [9]:
# Model evaluation function for Linear Regression
def evaluate_linear_regression(X_train, y_train, X_test, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"MSE: {mse:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

In [10]:
evaluate_linear_regression(X_train_final, y_train_filtered, X_test_final, y_test_filtered)

MSE: 339.4165 | RMSE: 18.4233 | MAE: 15.5927 | R²: 0.4647


In [11]:
# Yeo-Johnson power transformation to reduce skewness
skewed_feats = X_train_final[numerical_cols].apply(lambda x: x.skew()).sort_values(ascending=False)
high_skew = skewed_feats[abs(skewed_feats) > 0.5].index.tolist()

X_train_before_transform = X_train_final.copy()
X_test_before_transform = X_test_final.copy()

if len(high_skew) > 0:
    pt = PowerTransformer(method='yeo-johnson')
    X_train_final[high_skew] = pt.fit_transform(X_train_final[high_skew])
    X_test_final[high_skew] = pt.transform(X_test_final[high_skew])

    # palette = sns.color_palette("tab10", 2) 
    # num_cols = len(high_skew)
    # cols_per_row = 3
    # rows = (num_cols + cols_per_row - 1) // cols_per_row

    # fig, axes = plt.subplots(rows, cols_per_row, figsize=(6 * cols_per_row, 5 * rows))
    # axes = axes.flatten()

    # for i, col in enumerate(high_skew):
    #     sns.kdeplot(
    #         data=X_train_before_transform, x=col, fill=True, color=palette[1], alpha=0.3, label='Before (Capped)', ax=axes[i]
    #     )
    #     sns.kdeplot(
    #         data=X_train_final, x=col, fill=True, color=palette[0], alpha=0.7, label='After (Yeo-Johnson)', ax=axes[i]
    #     )
    #     axes[i].set_title(f'KDE for {col} (Before vs After)')
    #     axes[i].legend()

    # for j in range(i + 1, len(axes)):
    #     axes[j].set_visible(False)

    # plt.tight_layout()
    # plt.show()

else:
    print("No highly skewed numerical features found to transform.")


In [12]:
evaluate_linear_regression(X_train_final, y_train_filtered, X_test_final, y_test_filtered)

MSE: 343.7021 | RMSE: 18.5392 | MAE: 15.6946 | R²: 0.4579


In [13]:
# Scaling 
scaler = MinMaxScaler()
X_train_final[numerical_cols] = scaler.fit_transform(X_train_final[numerical_cols])
X_test_final[numerical_cols] = scaler.transform(X_test_final[numerical_cols])


In [14]:
evaluate_linear_regression(X_train_final, y_train_filtered, X_test_final, y_test_filtered)

MSE: 343.7021 | RMSE: 18.5392 | MAE: 15.6946 | R²: 0.4579


In [15]:
# Polynomial features or interaction terms can capture nonlinear relationships
poly_dict = {}
X_train_num = X_train[numerical_cols].copy()

for feature in numerical_cols:
    for p in range(2, 5):
        X_train_poly = X_train_num.copy()
        new_col_name = f"{feature}_pow_{p}"
        X_train_poly[new_col_name] = X_train_poly[feature] ** p

        lr = LinearRegression()
        lr.fit(X_train_poly, y_train)
        score = lr.score(X_train_poly, y_train) 

        poly_dict[score] = [feature, p]

best_score = max(poly_dict.keys())
best_feature, best_power = poly_dict[best_score]
print(f"Best polynomial feature: {best_feature}^{best_power} with R² score {best_score:.4f}")

poly_col_name = f"{best_feature}_pow_{best_power}"
X_train[poly_col_name] = X_train[best_feature] ** best_power
X_test[poly_col_name] = X_test[best_feature] ** best_power


Best polynomial feature: WHT^2 with R² score 0.6349


In [16]:
evaluate_linear_regression(X_train_final, y_train_filtered, X_test_final, y_test_filtered)

MSE: 343.7021 | RMSE: 18.5392 | MAE: 15.6946 | R²: 0.4579


In [ ]:
# Feature interactions
interaction_dict = {}
columns_list = X_train_final.columns.tolist() 

for feat1, feat2 in combinations(columns_list, 2):
    X_train_int = X_train_final.copy()
    new_col_name = f"{feat1}_x_{feat2}"

    X_train_int[new_col_name] = X_train_int[feat1] * X_train_int[feat2]

    model = LinearRegression()
    model.fit(X_train_int, y_train_filtered)

    y_pred = model.predict(X_train_int)
    r2 = r2_score(y_train_filtered, y_pred)

    interaction_dict[r2] = (feat1, feat2)

top_5_r2 = sorted(interaction_dict.keys(), reverse=True)[:5]

for r2 in top_5_r2:
    feat1, feat2 = interaction_dict[r2]
    col_name = f"{feat1}_x_{feat2}"
    
    X_train_final[col_name] = X_train_final[feat1] * X_train_final[feat2]
    X_test_final[col_name] = X_test_final[feat1] * X_test_final[feat2]

    print(f"Added interaction: {col_name} with R²: {r2:.4f}")

Added interaction: WHT_x_GND_Male with R²: 0.8678
Added interaction: WHT_x_GND_Female with R²: 0.8678
Added interaction: WHT_x_FRC with R²: 0.7457
Added interaction: WHT_x_SUS_Grauer's with R²: 0.6816
Added interaction: SUS_Grauer's_x_GND_Male with R²: 0.6760


In [18]:
evaluate_linear_regression(X_train_final, y_train_filtered, X_test_final, y_test_filtered)

MSE: 111.9190 | RMSE: 10.5792 | MAE: 8.4070 | R²: 0.8235


In [19]:
# Feature selection 
y_train_num = y_train_filtered.astype(float)
y_test_num = y_test_filtered.astype(float)

ridge_cv = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0, 100.0], cv=5)
lasso_cv = LassoCV(alphas=[0.01, 0.1, 1.0, 10.0], cv=5, max_iter=10000)

ridge_cv.fit(X_train_final, y_train_num)
lasso_cv.fit(X_train_final, y_train_num)

ridge_preds = ridge_cv.predict(X_test_final)
lasso_preds = lasso_cv.predict(X_test_final)

print(f"Best Ridge alpha: {ridge_cv.alpha_}")
print("Ridge Regression R2:", r2_score(y_test_num, ridge_preds))
print("Ridge Regression MSE:", mean_squared_error(y_test_num, ridge_preds))

print(f"\nBest Lasso alpha: {lasso_cv.alpha_}")
print("Lasso Regression R2:", r2_score(y_test_num, lasso_preds))
print("Lasso Regression MSE:", mean_squared_error(y_test_num, lasso_preds))

ridge_coefs = pd.Series(ridge_cv.coef_, index=X_train_final.columns).sort_values(key=abs, ascending=False)
lasso_coefs = pd.Series(lasso_cv.coef_, index=X_train_final.columns).sort_values(key=abs, ascending=False)

selected_features = lasso_coefs[lasso_coefs != 0].index.tolist()
print(f"Selected features from Lasso: {selected_features}")

Best Ridge alpha: 0.01
Ridge Regression R2: 0.8227175371004833
Ridge Regression MSE: 112.40905199029885

Best Lasso alpha: 0.01
Lasso Regression R2: 0.8209145298787628
Lasso Regression MSE: 113.55228031198651
Selected features from Lasso: ['WHT_x_GND_Female', 'GND_Female', "WHT_x_SUS_Grauer's", 'WHT', "SUS_Grauer's_x_GND_Male", 'WHT_x_FRC', "SUS_Grauer's", 'FRC', 'DSI', 'SUS_Western Lowland', 'AGE', 'SUS_Cross River', 'SUS_Mountain', 'GND_Male']


In [20]:
X_train_selected = X_train_final[selected_features]
X_test_selected = X_test_final[selected_features]

In [21]:
final_model = LinearRegression()
final_model.fit(X_train_selected, y_train_num)

y_pred = final_model.predict(X_test_selected)

# Evaluation metrics for regression
mse = mean_squared_error(y_test_num, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_num, y_pred)
r2 = r2_score(y_test_num, y_pred)

print(f"Final Model - MSE: {mse:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

Final Model - MSE: 111.9190 | RMSE: 10.5792 | MAE: 8.4070 | R²: 0.8235


#### Hyperparameter Tuning 

In [22]:
# Grid search
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 200],
    'fit_intercept': [True, False],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga'],
    'tol': [1e-4, 1e-3, 1e-2],
    'max_iter': [1000, 3000, 5000, 10000]
}


ridge = Ridge()
grid_search = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_selected, y_train_num)

print("Best negative MSE: {:.4f}".format(grid_search.best_score_))
print("Best parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_selected)

mse = mean_squared_error(y_test_num, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_num, y_pred)
r2 = r2_score(y_test_num, y_pred)

print(f"Test Set - MSE: {mse:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.0001; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.001; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.001; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.0001; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.01; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.01; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.01; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.01; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=True, max_iter=1000, solver=auto, tol=0.01; total time=   0.0s
[CV] END alpha=0.001, fit_intercept=Tr